# Vertical Profile Feature

The following notebook demonstrates the full workflow to access, process and visualize a vertical profile of ICON-CH1-EPS temperature data. 

<div style="text-align:center;">
  <img src="./images/t_2m_vertical_profile.png" style="width:70%;"/>
</div>

The data is retrieved using [Polytope](https://polytope.readthedocs.io/en/latest/), a feature extraction software developed by ECMWF. It applies concepts of computational geometry to extract n-dimensional polygons (also known as polytopes) from datacubes, such as a vertical profile. To access MeteoSwiss' operational ICON-CH1-EPS and ICON-CH2-EPS model data, [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) provides a wrapper around the polytope client that simplifies the request API. Follow the instructions to learn more about model data access via Polytope.

## Installation
Follow the instructions in [README.md](https://github.com/MeteoSwiss/nwp-fdb-polytope-demo/blob/main/README.md#Installation-1) to install the necessary dependencies.

## Configuring Access to Polytope
To use ICON CSCS polytope, you need a polytope offline token provided by MeteoSwiss. If you do not already have a token, you can request one [here](https://meteoswiss.atlassian.net/wiki/spaces/IW2/pages/327780397/Polytope#Offline-token-authentication).

In [ ]:
import os
import yaml

def load_config(path="config.yml"):
    if not os.path.exists(path):
        raise FileNotFoundError("Missing config.yml. Please create one based on config_example.yml.")
    with open(path, "r") as f:
        return yaml.safe_load(f)

config = load_config()

#ICON-CSCS Polytope credentials
os.environ["POLYTOPE_USER_KEY"] = config["meteoswiss"]["key"]
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-depl.mchml.cscs.ch"

## Selecting date and time of the forecast

The realtime FDB typically **includes only the most recent day of forecasts**. Therefore, it is necessary to specify the current date and select a corresponding forecast time in the past.

In [ ]:
from datetime import datetime, timedelta

# Current time
now = datetime.now()

# Subtract 12 hours
past_time = now - timedelta(hours=12)

# Round down to the nearest multiple of 6
rounded_hour = (past_time.hour // 6) * 6
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

# Format as YYYYMMDD and HHMM
date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date,time

## Define and rotate the polygon points

To use the polytope feature, it is necessary to rotate the given data point (here Zurich Airport) using a South Pole rotation with a reference of longitude 10° and latitude of -43°. 
> **IMPORTANT**: The function `transform_point()` expects first longitude and then latitude.

In [ ]:
import cartopy.crs as ccrs

# point for vertical profile
zrh_geo_point = (8.565074, 47.453928)  # (longitude, latitude) in WGS84

# South pole rotation of lon=10, latitude=-43
rotated_crs = ccrs.RotatedPole(
    pole_longitude=190, pole_latitude=43
)

# Convert a point from geographic to rotated coordinates
geo_crs = ccrs.PlateCarree()
rotated_point = rotated_crs.transform_point(zrh_geo_point[0], zrh_geo_point[1], geo_crs)

## Define the request

Once the data is rotated, we need to define a mars request using [meteodata-lab](https://polytope.readthedocs.io/en/latest/). The `feature` attribute allows you to extract **only the relevant data at the given points**. Thus, the amount of data that is retrieved from storage is significantly reduced. For the "vertical profile" `feature` the following dictionary is needed.
> **IMPORTANT**: Don't forget to specify that the data points are tuples that first contain longitude and then latitude.

In [ ]:
feature={
    "type" : "verticalprofile",
    "points" : [rotated_point],
    "axes" : "levelist"
}

Finally, we can define the request. This example fetches **2-m temperature** from **ICON-CH1-EPS** at the **pressure level** 100 to 850, for the **control forecast**, at the selected run date/time.
- `type="pf"` — perturbed member (requires `number`).
- `type="cf"` — control forecast (no `number`).

In [ ]:
from meteodatalab import mars

request = mars.Request(
    param="T",
    date=date,
    time=time,
    model=mars.Model.ICON_CH1_EPS,
    levtype=mars.LevType.PRESSURE_LEVEL,
    type="cf",
    step=0,
    levelist=range(100,900,50),
    feature=feature
)

## Data retrieval
Now we use [earthkit.data](https://earthkit-data.readthedocs.io/en/latest/) to load the data and convert it into an [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html).

In [ ]:
import earthkit.data as ekd
ds = ekd.from_source(
    "polytope",
    "mchgj",
    request.to_polytope(),
    stream=False
).to_xarray()

## Plotting

We use the library [earthkit.plots](https://earthkit-plots.readthedocs.io/en/latest/) to plot the data.

In [ ]:
from earthkit.plots.interactive import Chart

da = ds["t"].to_dataframe().reset_index()

chart = Chart()

# convert to degree Celsius
chart.line(x=da["t"].values - 273.15, y=da["levelist"].values)

# set axis names
chart.fig.update_layout(xaxis1={"title": "Temperature (°C)"})
chart.fig.update_layout(yaxis1={"title": "Pressure levels (hPa)"})
chart.fig.update_yaxes(autorange="reversed")

chart.title(f"Vertical Profile at Zurich Airport: Temperature (date={date}, time={time})")

chart.show()